In [4]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import xarray as xr

In [5]:
path = '/home/jatin/Downloads/sst1970-1979.nc'

ds = xr.open_dataset(path)
ds

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, time: 120)
Coordinates:
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float32 89.5 88.5 87.5 86.5 85.5 ... -86.5 -87.5 -88.5 -89.5
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1979-12-01T12:00:00
Data variables:
    sst      (time, lat, lon) float32 ...
Attributes:
    Conventions:                CF-1.0
    title:                      Monthly version of HadISST sea surface temper...
    institution:                Met Office, Hadley Centre for Climate Research
    source:                     HadISST
    history:                    09/11/2006 HadISST converted to NetCDF from p...
    references:                 Rayner, N. A., Parker, D. E., Horton, E. B., ...
    comment:                    Data restrictions: for academic research use ...
    supplementary_information:  Updates and supplementary information will be...
    contact:                    john.kennedy@metoffice.g

In [6]:
sst = ds['sst']

In [17]:
def month_to_season_combined(i, darray, *SEASONS):
    
    # month_to_season12 function
    def month_to_season12(xMon):
    
        # xMon is the DataArray passed as an argument

        # Create a new array of seasons
        season = np.array(["DJF","JFM","FMA","MAM","AMJ","MJJ","JJA","JAS","ASO","SON","OND","NDJ"])

        # Get the rank of dataarray
        len_of_dim = xMon.sizes
        rank_of_dim = len(len_of_dim)

        # Check if the rank of dim is valid or invalid; if invalid, exit the function
        if (rank_of_dim == 2 or rank_of_dim >= 5):
            print("month_to_season12: rank = {}".format(rank_of_dim))
            print("----- rank currently not handled -----")
            return None

        # Check if number of months are multiple of 12; if not, exit the function
        no_of_months = 12
        no_of_time = len_of_dim[xMon.dims[0]]
        if ((no_of_time % no_of_months) != 0):
            print("month_to_season12: dimension must be a multiple of 12")
            return None
        no_of_years = no_of_time/no_of_months

        # Store the size of latitude and longitude
        if (rank_of_dim >= 3):
            size_of_lat = len_of_dim[xMon.dims[rank_of_dim-2]]
            size_of_lon = len_of_dim[xMon.dims[rank_of_dim-1]]

        # Check if dimensions are named or not; if unnamed, exit the function
        for i in range(0, rank_of_dim):
            if xMon.dims[i] == None or xMon.dims[i] == "":
                print("mon_to_season12: All dimensions must be named")
                print("\t\tdimension {} is missing".format(i))
                return None

        # Calculating seasonal mean for rank = 1
        if (rank_of_dim == 1):

            # Calculate seasonal mean for each season
            dr = xMon.rolling(time = 3, center = True).mean(skipna = True)
            dr[0] = (xMon[0] + xMon[1]) * 0.5
            dr[(no_of_time-1)] = (xMon[(no_of_time-2)] + xMon[(no_of_time-1)]) * 0.5

            # Create a new DataArray using the existing ones
            xSea = xMon.copy(data = dr)

            # Add and update the attributes
            xSea.attrs['season'] = season
            xSea.attrs['long_name'] = "seasonal means: " + xSea.attrs['long_name']
            xSea = xSea.rename("xSea")

            # Return the newly created DataArray
            return xSea


        # Calculating seasonal mean for rank = 3
        if (rank_of_dim == 3):

            # Calculate seasonal mean for each season
            dr = xMon.rolling(time = 3, center = True).mean(skipna = True)
            dr[0,:,:] = (xMon[0,:,:] + xMon[1,:,:]) * 0.5
            dr[(no_of_time-1),:,:] = (xMon[(no_of_time-2),:,:] + xMon[(no_of_time-1),:,:]) * 0.5

            # Create a new DataArray using the existing ones
            xSea = xMon.copy(data = dr)

            # Add and update the attributes
            xSea.attrs['season'] = season
            xSea.attrs['long_name'] = "seasonal means: " + xSea.attrs['long_name']
            xSea = xSea.rename("xSea")
            # Return the newly created DataArray
            return xSea


        # Calculating seasonal mean for rank = 4
        if (rank_of_dim == 4):

            # Calculate seasonal mean for each season
            dr = xMon.rolling(time = 3, center = True).mean(skipna = True)
            dr[0,:,:,:] = (xMon[0,:,:,:] + xMon[1,:,:,:]) * 0.5
            dr[(no_of_time-1),:,:,:] = (xMon[(no_of_time-2),:,:,:] + xMon[(no_of_time-1),:,:,:]) * 0.5

            # Create a new DataArray using the existing ones
            xSea = xMon.copy(data = dr)

            # Add and update the attributes
            xSea.attrs['season'] = season
            xSea.attrs['long_name'] = "seasonal means: " + xSea.attrs['long_name']
            xSea = xSea.rename("xSea")
            # Return the newly created DataArray
            return xSea
    
    
    # month_to_season function
    def month_to_season(xMon, SEASON):
    
        # xMon is the DataArray and SEASON is the season passed as an argument

        # Create a new array of seasons
        season = np.array(["DJF","JFM","FMA","MAM","AMJ","MJJ","JJA","JAS","ASO","SON","OND","NDJ"])

        # Check if the season argument passed is valid or not
        if SEASON not in season:
            print("month_to_season: bad season: SEASON={}".format(SEASON))
            return None
        else:
            NMO_tup = np.where(season==SEASON)
            NMO = NMO_tup[0].item()

        # Get the rank of dataarray
        len_of_dim = xMon.sizes
        rank_of_dim = len(len_of_dim)

        # Check if the rank of dim is valid or invalid
        if (rank_of_dim == 2 or rank_of_dim >= 5):
            print("month_to_season12: rank = {}".format(rank_of_dim))
            print("----- rank currently not handled -----")
            return None

        # Check if number of months are multiple of 12
        no_of_months = 12
        no_of_time = len_of_dim[xMon.dims[0]]
        if ((no_of_time % no_of_months) != 0):
            print("month_to_season12: dimension must be a multiple of 12")
            return None


        # Store the size of latitude and longitude
        if (rank_of_dim >= 3):
            size_of_lat = len_of_dim[xMon.dims[rank_of_dim-2]]
            size_of_lon = len_of_dim[xMon.dims[rank_of_dim-1]]

        # Calculate number of years
        no_of_years = int(no_of_time/no_of_months)
        con = 1./3.

        # Starting and ending years
        no_year_start = 0
        no_year_last = no_of_years - 1
        if (NMO == 0):
            no_year_start = 1
        elif (NMO == (no_of_months-1)):
            no_year_last = no_of_years - 2

        # Create a copy of new dataarray using the existing one
        da_new = xMon.copy(deep = True, data = None)

        # For rank = 1
        if (rank_of_dim == 1):
            for nyr in range(no_year_start, (no_year_last+1)):
                n = nyr * no_of_months + NMO
                da_new[n] = (xMon[n-1] + xMon[n] + xMon[n+1]) * con

            if (NMO == 0):
                n = 0
                da_new[n] = (xMon[n] + xMon[n+1])*0.5

            if (NMO == (nmos-1)):
                n = (no_of_years-1)*no_of_months + NMO
                da_new[n] = (xMon[n] + xMon[n-1])*0.5

            da_new = da_new[NMO:no_of_time:no_of_months]               

        # For rank = 3
        if (rank_of_dim == 3):
            for nyr in range(no_year_start, (no_year_last+1)):
                n = nyr*no_of_months + NMO
                da_new[n,:,:] = (xMon[(n-1),:,:] + xMon[n,:,:] + xMon[(n+1),:,:]) * con

            if (NMO == 0):
                n = 0
                da_new[n,:,:] = (xMon[n,:,:] + xMon[(n+1),:,:])*0.5

            if (NMO == (no_of_months-1)):
                n = (no_of_years-1)*no_of_months + NMO
                da_new[n,:,:] = (xMon[n,:,:] + xMon[(n-1),:,:])*0.5

            da_new = da_new[NMO:no_of_time:no_of_months,:,:]


        # For rank = 4
        if (rank_of_dim == 4):
            for nyr in range(no_year_start, (no_year_last+1)):
                n = nyr*no_of_months + NMO
                da_new[n,:,:,:] = (xMon[(n-1),:,:,:] + xMon[n,:,:,:] + xMon[(n+1),:,:,:]) * con

            if (NMO == 0):
                n = 0
                da_new[0,:,:,:] = (xMon[n,:,:,:] + xMon[(n+1),:,:,:])*0.5

            if (NMO == (nmos-1)):
                n = (no_of_years-1)*no_of_months + NMO
                da_new[(n),:,:,:] = (xMon[n,:,:,:] + xMon[(n-1),:,:,:])*0.5

            da_new = da_new[NMO:no_of_time:no_of_months,:,:,:]


        # Add and update the attributes
        da_new = da_new.rename("x"+SEASON)
        da_new.attrs['NMO'] = NMO
        da_new.attrs['long_name'] = SEASON + ": " + da_new.attrs['long_name']

        # Return the newly created DataArray
        return da_new
    
    
    
    
    # month_to_seasonN function
    def month_to_seasonN(xMon, *SEASON):

        # xMon is the DataArray and *SEASON are the seasons passed as an argument

        # Create a new array of seasons
        season = np.array(["DJF","JFM","FMA","MAM","AMJ","MJJ","JJA","JAS","ASO","SON","OND","NDJ"])
        N = len(SEASON)

        # Check if the SEASON argument is present in season array
        for i in SEASON:
            if i not in season:
                print("month_to_seasonN: You have atleast one spelling error " + 
                      "in your SEASON specification. {} is not valid.".format(i))
                return None

        # Get the rank of dataarray
        len_of_dim = xMon.sizes
        rank_of_dim = len(len_of_dim)

        # Check if the rank of dim is valid or invalid; if invalid, exit the function
        if (rank_of_dim == 2 or rank_of_dim >= 5):
            print("month_to_season12: rank = {}".format(rank_of_dim))
            print("----- rank currently not handled -----")
            return None

        # Check if number of months are multiple of 12; if not, exit the function
        no_of_months = 12
        no_of_time = len_of_dim[xMon.dims[0]]
        if ((no_of_time % no_of_months) != 0):
            print("month_to_season12: dimension must be a multiple of 12")
            return None
        no_of_years = no_of_time/no_of_months

        # Call the month_to_season12 function
        xSea12 = month_to_season12(xMon)


        # Store the size of latitude and longitude
        if (rank_of_dim >= 3):
            size_of_lat = len_of_dim[xMon.dims[rank_of_dim-2]]
            size_of_lon = len_of_dim[xMon.dims[rank_of_dim-1]]

        # Check the index of first SEASON passed in season array
        i = SEASON[0]
        NMO1 = np.where(season==i)[0].item()

        xSeaN = xSea12.copy(data = None, deep = True)

        # Store the SEASONS in a numpy array
        sea = np.array([], dtype ='int')
        sea_code = np.array([], dtype = 'str')

        for n in range(0, N):
            NMO = np.where(season==SEASON[n])[0].item()
            sea = np.append(sea, NMO)
            sea_code = np.append(sea_code, season[NMO])

        # Create a new dataarray to store season co-ordinate data
        new_da = xr.DataArray(data = sea,dims = 'season', name='season')

        # Calculations for rank = 1
        if (rank_of_dim == 1):
            for ns in range(0,N):
                NMO = np.where(season==SEASON[ns])[0].item()
                if(NMO == sea[0]):
                    xSeaN = xSea12[NMO:no_of_time:no_of_months]
                else:
                    xS = xSea12[NMO:no_of_time:no_of_months]
                    # Concat the data and pass the season co-ordinate
                    xSeaN = xr.concat([xSeaN, xS], dim = new_da)

        # Calculating seasonal mean for rank = 3
        if (rank_of_dim == 3):
            for ns in range(0,N):
                NMO = np.where(season==SEASON[ns])[0].item()
                if (NMO == sea[0]):
                    xSeaN = xSea12[NMO:no_of_time:no_of_months,:,:]
                else:
                    xS = xSea12[NMO:no_of_time:no_of_months,:,:]
                    xSeaN = xr.concat([xSeaN, xS], dim = new_da)

        # Calculating seasonal mean for rank = 4
        if (rank_of_dim == 4):
            for ns in range(0,N):
                NMO = np.where(season==SEASON[ns])[0].item()
                if (NMO == sea[0]):
                    xSeaN = xSea12[NMO:no_of_time:no_of_months,:,:,:]
                else:
                    xS = xSea12[NMO:no_of_time:no_of_months,:,:,:]
                    xSeaN = xr.concat([xSeaN, xS], dim = new_da)


            # Add and update the attributes
        if(xSeaN.attrs['long_name'] or xSeaN.attrs['description'] or xSeaN.attrs['standard_name']):
            xSeaN.attrs['long_name'] = "Seasonal Means: " + xSeaN.attrs['long_name']

            # Since the season co-ordinate contains indexes of data, we replace 
            # with the actual season string.
            xSeaN['season'] = sea_code

            # Return the newly created DataArray
            return xSeaN

    
    
    if (i == 1):
        print(month_to_season12(darray))
    elif (i == 2):
        print(month_to_season(darray, *SEASONS))
    elif (i == 3):
        print(month_to_seasonN(darray, *SEASONS))
    else:
        print("Invalid Input")

In [19]:
month_to_season_combined(3, sst, "DJF", "SON")

<xarray.DataArray 'xSea' (season: 2, time: 20, lat: 180, lon: 360)>
array([[[[-1000., ..., -1000.],
         ...,
         [   nan, ...,    nan]],

        ...,

        [[   nan, ...,    nan],
         ...,
         [   nan, ...,    nan]]],


       [[[   nan, ...,    nan],
         ...,
         [   nan, ...,    nan]],

        ...,

        [[-1000., ..., -1000.],
         ...,
         [   nan, ...,    nan]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1970-01-01T12:00:00 ... 1979-10-01T12:00:00
  * lon      (lon) float32 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat      (lat) float32 89.5 88.5 87.5 86.5 85.5 ... -86.5 -87.5 -88.5 -89.5
  * season   (season) <U3 'DJF' 'SON'
Attributes:
    long_name:     Seasonal Means: seasonal means: Monthly 1 degree resolutio...
    units:         degrees C
    actual_range:  [-1000.         33.53844]
    description:   HadISST sea surface temperature. values of -1000 indicate ...
    season:        ['DJF' 'JFM'